In [1]:
## Load relevant R packages.

inLibraries = list('repr','spdep','raster','rgdal','rgeos')
for (rpack in inLibraries) {
  if (is.element(rpack,installed.packages()[,1])){
      #Load the library into R
      suppressMessages(library(rpack,character.only = TRUE))
    }
    else {
        print(paste("Warning:  ",rpack," is not an installed package"))
    }
}

##  Resize plot area.

options(repr.plot.width=6, repr.plot.height=6)

library(geojsonio)
library(dplyr)


Registered S3 method overwritten by 'geojsonsf':
  method        from   
  print.geojson geojson


Attaching package: ‘geojsonio’


The following object is masked from ‘package:base’:

    pretty



Attaching package: ‘dplyr’


The following objects are masked from ‘package:rgeos’:

    intersect, setdiff, union


The following objects are masked from ‘package:raster’:

    intersect, select, union


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
hist_urban <- read.csv("data/urban/urbanpop.csv",stringsAsFactors=FALSE)

coordinates(hist_urban)<- ~Longitude+Latitude

st_crs(hist_urban, 4326)

Coordinate Reference System: NA

In [3]:
writeOGR(hist_urban, "data/results/", "hist_urban_sf", driver = "ESRI Shapefile")

ERROR: Error in writeOGR(hist_urban, "data/results/", "hist_urban_sf", driver = "ESRI Shapefile"): layer exists, use a new layer name


In [4]:
meteorites <- read.csv("data/results/meteorites.csv")

coordinates(meteorites)=~longitude+latitude

st_crs(meteorites, 4326)

Coordinate Reference System: NA

In [ ]:
writeOGR(meteorites, "data/results/", "meteorites_sf", driver = "ESRI Shapefile")

In [5]:
head(meteorites)

,X,name,id,nametype,recclass,mass_in_grams,fell_or_found,year,GeoLocation,group_name,Chondrite.Achondrite,type,L3,L4
,<int>,<chr>,<int>,<chr>,<chr>,<dbl>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,5625,Aachen,1,Valid,L5,21,Fell,1880,"(50.775, 6.08333)",L,Chondrite,Stony,Ordinary,L
2,568,Aarhus,2,Valid,H6,720,Fell,1951,"(56.18333, 10.23333)",H,Chondrite,Stony,Ordinary,H
3,916,Abajo,4,Valid,H5,331,Found,1982,"(26.8, -105.41667)",H,Chondrite,Stony,Ordinary,H
4,918,Abbott,5,Valid,H3-6,21100,Found,1951,"(36.3, -104.28333)",H,Chondrite,Stony,Ordinary,H
5,399,Abee,6,Valid,EH4,107000,Fell,1952,"(54.21667, -113.0)",EH,Chondrite,Stony,Enstatite,EH-EL
6,6004,Abernathy,7,Valid,L6,2914,Found,1941,"(33.85, -101.8)",L,Chondrite,Stony,Ordinary,L


In [7]:
i<-1 

meteorite_sample <- (meteorites[i,])                # Make a variable with just the line of the meteorite under study
sample_year <- meteorite_sample@data$year           # Get the year of impact/find

meteorite_sample.buf <- gBuffer(meteorite_sample,   # Create a .25 degree buffer around the impact/find site.
                                byid=TRUE, 
                                id=NULL, 
                                width=.25, 
                                quadsegs=5, 
                                capStyle="ROUND", 
                                joinStyle="ROUND", 
                                mitreLimit=1.0)

hist_urban_sample <- hist_urban[hist_urban$year < sample_year,]         #pull the pop data for the 500 years before impact/find
hist_urban_sample <- hist_urban_sample[hist_urban_sample$year > (sample_year-500),]

selPts <- overGeomGeom(meteorite_sample.buf, hist_urban_sample, returnList = TRUE, fn = NULL)    # Get the cities that are inside the bufffer
intPtSet = unlist(selPts)
intersect <- hist_urban_sample[intPtSet,]

if(length(intersect) != 0){                           # If a city was found, find the one closest in time to current
    closest_year <- max(intersect@data$year)
    sample_city <- intersect@data$City[intersect@data$year == closest_year]
    sample_population <- intersect@data$pop[intersect@data$year == closest_year]
    } else {                                          # If no city was found, put NAs in the table
    closest_year <- NA
    sample_city <- NA
    sample_population <- NA
    }


population_table <- data.frame(coordinates(meteorite_sample),
                               meteorite_sample$name, 
                               meteorite_sample$id,
                               meteorite_sample$year,
                               sample_city,
                               sample_population,
                               closest_year)

names(population_table) <- c("x", "y", "name", "id", "year","closest_city","pop","pop_year")


# There's probably a better way to do this, but at the time, it made sense to 
# run it through once to create the table,then start the loop with the 2nd entry.

l <- length(meteorites)

for (i in 2:l){
    meteorite_sample <- (meteorites[i,])                # Make a variable with just the line of the meteorite under study
    sample_year <- meteorite_sample@data$year           # Get the year of impact/find

    meteorite_sample.buf <- gBuffer(meteorite_sample,   # Create a .25 degree buffer around the impact/find site.
                                    byid=TRUE, 
                                    id=NULL, 
                                    width=.25, 
                                    quadsegs=5, 
                                    capStyle="ROUND", 
                                    joinStyle="ROUND", 
                                    mitreLimit=1.0)

    hist_urban_sample <- hist_urban[hist_urban$year < sample_year,]         #pull the pop data for the 500 years before impact/find
    hist_urban_sample <- hist_urban_sample[hist_urban_sample$year > (sample_year-500),]

    selPts <- overGeomGeom(meteorite_sample.buf, hist_urban_sample, returnList = TRUE, fn = NULL)    # Get the cities that are inside the bufffer
    intPtSet = unlist(selPts)
    intersect <- hist_urban_sample[intPtSet,]

    if(length(intersect) != 0){                           # If a city was found, find the one closest in time to current
        closest_year <- max(intersect@data$year)
        sample_city <- intersect@data$City[intersect@data$year == closest_year]
        sample_population <- intersect@data$pop[intersect@data$year == closest_year]
        } else {                                          # If no city was found, put NAs in the table
        closest_year <- NA
        sample_city <- NA
        sample_population <- NA
        }

    new_population_table <- data.frame(coordinates(meteorite_sample),
            write.csv(meteorite_LCs,'data/results/meteorites_landcover.csv')                   meteorite_sample$name, 
                               meteorite_sample$id,
                               meteorite_sample$year,
                               sample_city,
                               sample_population,
                               closest_year)

    names(new_population_table) <- c("x", "y", "name", "id", "year","closest_city","pop","pop_year")
    population_table <- rbind(population_table, new_population_table)
}

Warning message in data.frame(coordinates(meteorite_sample), meteorite_sample$name, :
“row names were found from a short variable and have been discarded”
Warning message in data.frame(coordinates(meteorite_sample), meteorite_sample$name, :
“row names were found from a short variable and have been discarded”
Warning message in data.frame(coordinates(meteorite_sample), meteorite_sample$name, :
“row names were found from a short variable and have been discarded”
Warning message in data.frame(coordinates(meteorite_sample), meteorite_sample$name, :
“row names were found from a short variable and have been discarded”
Warning message in data.frame(coordinates(meteorite_sample), meteorite_sample$name, :
“row names were found from a short variable and have been discarded”
Warning message in data.frame(coordinates(meteorite_sample), meteorite_sample$name, :
“row names were found from a short variable and have been discarded”
Warning message in data.frame(coordinates(meteorite_sample), meteorite

In [8]:
head(population_table)

,x,y,name,id,year,closest_city,pop,pop_year
,<dbl>,<dbl>,<chr>,<int>,<int>,<chr>,<int>,<int>
1,6.08333,50.77500,Aachen,1,1880,Aachen,48688,1851
2,10.23333,56.18333,Aarhus,2,1951,Aarhus,51000,1900
3,-105.41667,26.80000,Abajo,4,1982,NA,NA,NA
4,-104.28333,36.30000,Abbott,5,1951,NA,NA,NA
5,-113.00000,54.21667,Abee,6,1952,NA,NA,NA
6,-101.80000,33.85000,Abernathy,7,1941,NA,NA,NA


In [9]:
write.csv(population_table,'data/results/meteorites_population.csv')